In [ ]:
!pip install openai

In [ ]:
!pip install llama-index
!pip install pypdf

In [ ]:
%pip install llama-index-readers-google

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
## Retrieval augmented generation

import os

In [ ]:
api_key = r'apikey'

In [ ]:
os.environ['OPENAI_API_KEY'] = api_key

In [ ]:
## test open ai api key

from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
  model="gpt-3.5-turbo-0125",
  response_format={ "type": "json_object" },
  messages=[
    {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
    {"role": "user", "content": "Who won the world series in 2020?"}
  ]
)
print(response.choices[0].message.content)

{
  "winner": "Los Angeles Dodgers",
  "year": 2020
}


In [ ]:
from llama_index.core import VectorStoreIndex,SimpleDirectoryReader
from llama_index.readers.google import GoogleDriveReader

In [ ]:
reader = SimpleDirectoryReader(input_dir="/content/data/")

In [ ]:
documents = reader.load_data()

In [ ]:
documents

[Document(id_='d78c3e7f-169f-4a6a-9b2e-4f09b9c3ba56', embedding=None, metadata={'page_label': '1', 'file_name': 'Prompt-Engineering-Lecture-Elvis (1).pdf', 'file_path': '/content/data/Prompt-Engineering-Lecture-Elvis (1).pdf', 'file_type': 'application/pdf', 'file_size': 3164942, 'creation_date': '2024-05-07', 'last_modified_date': '2024-05-07'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Elvis SaraviaPrompt EngineeringA lecture by DAIR.AI', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 Document(id_='19c9e9e6-47c6-44fe-9a8a-9e1dae1bb80f', embedding=None, metadata={'page_label': '2', 'file_name': 'Prompt-Engineering-Lecture-Elvis (1).pdf', 'file_path

In [ ]:
index=VectorStoreIndex.from_documents(documents,show_progress=True)

Parsing nodes:   0%|          | 0/50 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
index

In [ ]:
query_engine=index.as_query_engine()

In [ ]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

retriever=VectorIndexRetriever(index=index,similarity_top_k=4)
postprocessor=SimilarityPostprocessor(similarity_cutoff=0.80)

query_engine=RetrieverQueryEngine(retriever=retriever,
                                  node_postprocessors=[postprocessor])

In [ ]:
response=query_engine.query("how to do a few shot prompt and show a sample")

In [ ]:
from llama_index.core.response.pprint_utils import pprint_response
pprint_response(response,show_source=True)
print(response)

Final Response: To do a few-shot prompt, you can provide exemplars in
prompts to guide the model towards better performance. For example,
you can present a set of numbers and ask whether the odd numbers in
the group add up to an even number.
______________________________________________________________________
Source Node 1/2
Node ID: 129359c4-6de3-4dae-a8b3-64ca57492e80
Similarity: 0.8095783853057031
Text: Few-shot Prompts•Few-shot prompting allows us to provide
exemplars in prompts to steer the model towards better performanceThe
odd numbers in this group add up to an even number: 4, 8, 9, 15, 12,
2, 1.A: The answer is False.The odd numbers in this group add up to an
even number: 17,  10, 19, 4, 8, 12, 24.A: The answer is True.The odd
numbers in ...
______________________________________________________________________
Source Node 2/2
Node ID: ff3329da-dd17-4125-b2b0-04b0f98cd780
Similarity: 0.8040738545468594
Text: Prompt Engineering Techniques•Many advanced prompting techniques
ha